In [37]:
import json
import pyproj
import numpy

proj = pyproj.Transformer.from_crs(4326, 3395, always_xy=False)

def normalize_v3(arr):
    ''' Normalize a numpy array of 3 component vectors shape=(n,3) '''
    lens = numpy.sqrt( arr[:,0]**2 + arr[:,1]**2 + arr[:,2]**2 )
    arr[:,0] /= lens
    arr[:,1] /= lens
    arr[:,2] /= lens                
    return arr

In [2]:
ff = json.loads(open('park_slope.json').read())

In [115]:
buildings = {}
centroid = [0,0]
for layer in ff['layers']:
    if layer['id'] == 'buildings':
        coordinates = []
        indices = []
        nverts = 0
        centroid = [0,0]
        size=0
        for pp in layer['data']:
            coord = pp['geometry']['coordinates']
            for i in range(0,len(coord),3):
                x,y = proj.transform(coord[i],coord[i+1])
                centroid[0]+=x
                centroid[1]+=y
                size+=1
        centroid = [centroid[0]/size,centroid[1]/size,0]
        print(centroid)
            
        for pp in layer['data']:
            coord = pp['geometry']['coordinates']
            ind = pp['geometry']['indices']
            size = len(coord)
            summ = [0,0]
            for i in range(0,len(coord),3):
                x,y = proj.transform(coord[i],coord[i+1])
                coordinates.append([x-centroid[0],y-centroid[1],coord[i+2]])
            
            for i in range(0,len(ind),3):
                globalindex0 = int(ind[i]+nverts)
                globalindex1 = int(ind[i+1]+nverts)
                globalindex2 = int(ind[i+2]+nverts)
                indices.append([globalindex0,globalindex1,globalindex2])
            nverts+=(size/3)
        
        coordinates = numpy.array(coordinates)
        indices = numpy.array(indices)
        norm = numpy.zeros( coordinates.shape, dtype=coordinates.dtype )
        tris = coordinates[indices]
        n = numpy.cross( tris[::,1 ] - tris[::,0]  , tris[::,2 ] - tris[::,0] )
        normalize_v3(n)
        norm[ indices[:,0] ] += n
        norm[ indices[:,1] ] += n
        norm[ indices[:,2] ] += n
        norm = normalize_v3(norm)
        norm = numpy.nan_to_num(norm)
        
        buildings['coordinates'] = coordinates.flatten().tolist()
        buildings['indices'] = indices.flatten().tolist()
        buildings['normals'] = norm.flatten().tolist()
        buildings['color'] = [0.5,0.6,0.9, 1]

[-8235610.987445664, 4935998.989271286, 0]


In [112]:
water = {}
for layer in ff['layers']:
    if layer['id'] == 'water':
        coordinates = []
        indices = []
        nverts = 0
        for pp in layer['data']:
            coord = pp['geometry']['coordinates']
            ind = pp['geometry']['indices']
            size = len(coord)
            for i in range(0,len(coord),2):
                x,y = proj.transform(coord[i],coord[i+1])
                coordinates.append(x-centroid[0])
                coordinates.append(y-centroid[1])
                coordinates.append(1)
            for i in range(0,len(ind),3):
                globalindex0 = int(ind[i]+nverts)
                globalindex1 = int(ind[i+1]+nverts)
                globalindex2 = int(ind[i+2]+nverts)
                # invert normal
                indices.append(globalindex0)
                indices.append(globalindex2)
                indices.append(globalindex1)
            nverts+=(size/2)
        water['coordinates'] = coordinates
        water['indices'] = indices
        water['color'] = [185/255,205/255,210/255,1.0]

In [111]:
park = {}
for layer in ff['layers']:
    if layer['id'] == 'parks':
        coordinates = []
        indices = []
        nverts = 0
        for pp in layer['data']:
            coord = pp['geometry']['coordinates']
            ind = pp['geometry']['indices']
            size = len(coord)
            for i in range(0,len(coord),2):
                x,y = proj.transform(coord[i],coord[i+1])
                coordinates.append(x-centroid[0])
                coordinates.append(y-centroid[1])
                coordinates.append(2)
            for i in range(0,len(ind),3):
                globalindex0 = int(ind[i]+nverts)
                globalindex1 = int(ind[i+1]+nverts)
                globalindex2 = int(ind[i+2]+nverts)
                # invert normal
                indices.append(globalindex0)
                indices.append(globalindex2)
                indices.append(globalindex1)
            nverts+=(size/2)
        park['coordinates'] = coordinates
        park['indices'] = indices
        park['color'] = [195/255,208/255,178/255,1.0]

In [110]:
surface = {}
for layer in ff['layers']:
    if layer['id'] == 'surface':
        coordinates = layer['data'][0]['geometry']['coordinates']
        ind = layer['data'][0]['geometry']['indices']
        indices = []
        aux = []
        for i in range(0,len(coordinates),2):
            x,y = proj.transform(coordinates[i],coordinates[i+1])
            aux.append(x-centroid[0])
            aux.append(y-centroid[1])
            aux.append(-5)
        for i in range(0,len(ind),3):
            index0 = int(ind[i])
            index1 = int(ind[i+1])
            index2 = int(ind[i+2])
            # invert normal
            indices.append(index0)
            indices.append(index2)
            indices.append(index1)
        surface['coordinates'] = aux
        surface['indices'] = indices
        surface['color'] = [238/255,238/255,238/255,1.0]

In [116]:
out = {}
out['buildings'] = buildings
out['water'] = water
out['parks'] = park
out['surface'] = surface
with open('city.json', 'w') as f:
    json.dump(out, f)